# Imports

# Synthetic Dataset Creation

In [ ]:
import numpy as np
import pandas as pd

# Reproducible generator for synthetic samples
rng = np.random.default_rng(42)
n_samples = 200

materials = rng.integers(0, 3, size=n_samples)
temperatures = rng.integers(200, 1001, size=n_samples)
exposure_time = rng.integers(1, 61, size=n_samples)
environment_factor = rng.uniform(0.8, 1.2, size=n_samples)

df = pd.DataFrame(
    {
        'material_type': materials,
        'temperature_c': temperatures,
        'exposure_time_min': exposure_time,
        'environment_factor': environment_factor,
    }
)

df['thermal_load'] = df['temperature_c'] * df['exposure_time_min']
df['severity_index'] = df['thermal_load'] * df['environment_factor']

material_weight = {0: 0.9, 1: 1.1, 2: 1.0}
base_risk = df['severity_index'] / df['severity_index'].max()
noise = rng.normal(0, 0.05, size=n_samples)
df['risk_score'] = 100 * (base_risk * df['material_type'].map(material_weight) + noise)
df['risk_score'] = df['risk_score'].clip(0, 100)

df.head()


# Feature Engineering

# Model Training

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

feature_cols = [
    'material_type',
    'temperature_c',
    'exposure_time_min',
    'environment_factor',
    'thermal_load',
    'severity_index',
]
X = df[feature_cols]
y = df['risk_score']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

print(f"Train R^2: {model.score(X_train, y_train):.2f}")
print(f"Test R^2: {model.score(X_test, y_test):.2f}")


# Inference and Output Interpretation

In [ ]:
# Example inference input
example = {
    'material_type': 1,
    'temperature_c': 650,
    'exposure_time_min': 20,
    'environment_factor': 1.05,
}
example['thermal_load'] = example['temperature_c'] * example['exposure_time_min']
example['severity_index'] = example['thermal_load'] * example['environment_factor']

example_df = pd.DataFrame([example])
predicted_score = float(model.predict(example_df)[0])

if predicted_score <= 33:
    risk_class = 'Low'
elif predicted_score <= 66:
    risk_class = 'Medium'
else:
    risk_class = 'High'

resistance_index = 100 - predicted_score

results = {
    'input': example,
    'risk_score': round(predicted_score, 2),
    'risk_class': risk_class,
    'resistance_index': round(resistance_index, 2),
}

results
